In [1]:
from urllib import quote_plus
import urllib2,cookielib
import regex
import codecs
import lxml.etree as ET
import regex
from nltk.tokenize import sent_tokenize
import nltk
from multiprocessing.dummy import Pool as ThreadPool

import pprint
pp = pprint.PrettyPrinter(indent=4)
print "done"

done


In [2]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\shachar\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
### pick a word here ###
searchedWord = "big avocado"

# pull data from google
def calculateParallel(func, numbers, threads=2):
    pool = ThreadPool(threads)
    results = pool.map(func, numbers)
    pool.close()
    pool.join()
    return results

def makeReq(i):
    bracetedRequest = "\""+searchedWord+"\" filetype:pdf"

    site = "https://www.google.co.il/search?q=" + quote_plus(bracetedRequest) + "&start=" + str(i*10)
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
           'Accept-Encoding': 'none',
           'Accept-Language': 'en-US,en;q=0.8',
           'Connection': 'keep-alive'}

    req = urllib2.Request(site, headers=hdr)

    try:
        page = urllib2.urlopen(req)
    except urllib2.HTTPError, e:
        print e.fp.read()
    
    data = page.read()
    print "received:",len(data)
    return data

print "start"
results = calculateParallel(makeReq,range(10),10)

content = " ".join(results)
print "done!"


start
received: 273914
received: 274264
received: 274526
received: 276268
received: 275058
received: 275249
received: 276607
received: 275784
received: 275836
received: 276497
done!


In [4]:
# parse
text = content

text = regex.sub("\[http[^]]+? ([^]]+)]", r"\1", text) 
text = regex.sub("\[http[^]]+]", "", text) 
text = regex.sub("(?s)<script.+?</script>", "", text) # remove scripts 
text = regex.sub("(?s)<style.+?</style>", "", text) # remove scripts 
text = regex.sub("(?s)<ref>.+?</ref>", "", text) # remove reference links
text = regex.sub("(?s)<[^>]+>", "", text) # remove html tags
text = regex.sub("&[a-z]+;", "", text) # remove html entities
text = regex.sub("(?s){{.+?}}", "", text) # remove markup tags
text = regex.sub("(?s){.+?}", "", text) # remove markup tags
text = regex.sub("(?s)\[\[([^]]+\|)", "", text) # remove link target strings
text = regex.sub("(?s)\[\[([^]]+\:.+?]])", "", text) # remove media links

text = regex.sub("[']{5}", "", text) # remove italic+bold symbols
text = regex.sub("[']{3}", "", text) # remove bold symbols
text = regex.sub("[']{2}", "", text) # remove italic symbols

text = regex.sub(u"[^ \r\n\p{latin}}\d'.?!]", " ", text)
text = regex.sub(r'[^\x00-\x7F]+',' ', text)
text = text.lower()

text = regex.sub("[ ]{2,}", " ", text) # Squeeze spaces.

paras = text.split("\n")
final = ""
for para in paras:
    if len(para) > 500:
        sents = [regex.sub("([.!?]+$)", r" \1", sent) for sent in sent_tokenize(para.strip())]
        final += " ".join(sents) + "\n"

print "ended up with text of length",len(final)

ended up with text of length 30513


In [6]:
# sort by relevance
raw = regex.findall(u"[^\s\d\w][\s\w]*"+ searchedWord +"[\s\w]*[^\s\d\w]", text)

raw = [x[1:] for x in raw if len(x) <200 and len(x) > 30 and "www" not in x and not any(char.isdigit() for char in x) and "pdf" not in x]
raw = list(set(raw))

#pp.pprint(raw)
ranked = [(float([x for (_,x) in nltk.pos_tag(y.split())].count('NN'))/len([x for (_,x) in nltk.pos_tag(y.split())]),y) for y in raw ]
pp.pprint(sorted(ranked, key=lambda tup: abs(0.1-tup[0])))


[   (   0.09523809523809523,
        ' back to manila enroute make a stop at fruits stalls where you can pickup some mangoes big avocado and fresh strawberries.'),
    (   0.1111111111111111,
        ' enroute make a stop at fruits stalls where you can pickup some mangoes big avocado and fresh strawberries.'),
    (   0.16666666666666666,
        ' big avocado nor do they have a particularly apocalyptic sense of reality.'),
    (   0.2222222222222222,
        ' america was growing like your big avocado is growing.'),
    (   0.23076923076923078,
        ' here are some facts about seeds we will consider big avocado apricot peach.'),
    (   0.25,
        ' but the peace with dignity group said such payments were levied only on part of the population such as big avocado plantations and were help ful in some.'),
    (   0.2857142857142857,
        ' down the big avocado tree drops to the ground and heads right for me.'),
    (   0.29411764705882354,
        'has big avocado tree in centr